# 🚀 Day 2: Thread Indexing Mastery

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sdodlapa/cuda-lab/blob/main/learning-path/week-01/day-2-thread-indexing.ipynb)

> **Note:** If running on Google Colab, go to `Runtime → Change runtime type → T4 GPU` before starting!

## 📋 This Notebook Structure

| Section | Language | Purpose |
|---------|----------|---------|
| **Part 1** | 🔷 **CUDA C++** | Primary - Industry standard |
| **Part 2** | 🐍 Python (Numba) | Optional - Quick prototyping |

**Focus on Part 1 (CUDA C++)** - Python section is for reference only.

---

# Part 1: CUDA C++ (Primary) 🔷

## Verify CUDA Installation

In [ ]:
# Verify CUDA is available
!nvcc --version
print("\n" + "="*50)
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

## Learning Objectives

Today you'll master **thread indexing** - how each thread knows which data element to process.

- Understand 1D, 2D, and 3D thread indexing
- Master the relationship between blocks, threads, and global indices
- Implement grid-stride loops for handling any array size
- Apply indexing to real 2D problems (matrices, images)

## Thread Hierarchy

```
                           GRID
                ┌─────────────────────────┐
                │ Block   Block   Block   │
                │ (0,0)   (1,0)   (2,0)   │
                │                         │
                │ Block   Block   Block   │
                │ (0,1)   (1,1)   (2,1)   │
                └─────────────────────────┘
                        
                      Each Block:
                ┌─────────────────────────┐
                │ Thread Thread Thread    │
                │ (0,0)  (1,0)  (2,0)     │
                │                         │
                │ Thread Thread Thread    │
                │ (0,1)  (1,1)  (2,1)     │
                └─────────────────────────┘
```

**CUDA C++ Built-in Variables:**
- `threadIdx.x/y/z` - Thread index within block
- `blockIdx.x/y/z` - Block index within grid  
- `blockDim.x/y/z` - Threads per block
- `gridDim.x/y/z` - Blocks in grid

---

## 1. 1D Indexing: The Foundation

For 1D arrays, each thread needs a unique **global index**:

```cpp
int idx = blockIdx.x * blockDim.x + threadIdx.x;
```

```
Example: 3 blocks × 4 threads/block = 12 threads

Block 0:  Thread 0  Thread 1  Thread 2  Thread 3
          idx=0     idx=1     idx=2     idx=3
          
Block 1:  Thread 0  Thread 1  Thread 2  Thread 3  
          idx=4     idx=5     idx=6     idx=7
          
Block 2:  Thread 0  Thread 1  Thread 2  Thread 3
          idx=8     idx=9     idx=10    idx=11
```

In [ ]:
%%writefile thread_indexing_1d.cu
/**
 * Day 2: 1D Thread Indexing Demo
 * 
 * Each thread prints its indices to understand the hierarchy.
 */

#include <stdio.h>
#include <cuda_runtime.h>

__global__ void printIndices() {
    // Calculate global thread index
    int globalIdx = blockIdx.x * blockDim.x + threadIdx.x;
    
    printf("Block %d, Thread %d → Global Index %d\n",
           blockIdx.x, threadIdx.x, globalIdx);
}

__global__ void vectorAdd(float *a, float *b, float *c, int n) {
    // The fundamental indexing pattern
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // ALWAYS check bounds!
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

int main() {
    printf("=== 1D Thread Indexing Demo ===\n\n");
    
    // Demo 1: Print thread indices
    printf("Launch: 3 blocks × 4 threads/block\n");
    printf("---------------------------------\n");
    printIndices<<<3, 4>>>();
    cudaDeviceSynchronize();
    
    // Demo 2: Vector addition
    printf("\n=== Vector Addition ===\n");
    int n = 10;
    size_t size = n * sizeof(float);
    
    float h_a[] = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};
    float h_b[] = {10, 20, 30, 40, 50, 60, 70, 80, 90, 100};
    float h_c[10];
    
    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);
    
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);
    
    // Calculate grid size with ceiling division
    int threadsPerBlock = 4;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    
    printf("Array size: %d\n", n);
    printf("Threads per block: %d\n", threadsPerBlock);
    printf("Blocks needed: %d\n", blocksPerGrid);
    printf("Total threads: %d (some won't do work)\n\n", blocksPerGrid * threadsPerBlock);
    
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);
    cudaDeviceSynchronize();
    
    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);
    
    printf("Results: ");
    for (int i = 0; i < n; i++) {
        printf("%.0f ", h_c[i]);
    }
    printf("\n");
    
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 thread_indexing_1d.cu -o thread_indexing_1d && ./thread_indexing_1d

### Key Observations

1. **Thread order is NOT guaranteed** - Blocks/threads may execute in any order
2. **Bounds checking is essential** - When n=10 and total threads=12, threads 10 and 11 must NOT access array
3. **Ceiling division formula**: `(n + blockSize - 1) / blockSize` ensures we cover all elements

---

## 2. Grid-Stride Loops: Professional Pattern

When array size >> thread count, use a **grid-stride loop**:

```cpp
// Each thread processes MULTIPLE elements
for (int i = idx; i < n; i += gridDim.x * blockDim.x) {
    // Process element i
}
```

This pattern:
- ✅ Works for ANY array size
- ✅ Reuses threads efficiently  
- ✅ Industry standard pattern

In [ ]:
%%writefile grid_stride.cu
/**
 * Grid-Stride Loop Pattern
 * 
 * The professional way to handle arrays of ANY size.
 */

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Naive approach: 1 thread = 1 element (requires enough threads)
__global__ void vectorAddNaive(float *a, float *b, float *c, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

// Grid-stride loop: each thread handles MULTIPLE elements
__global__ void vectorAddGridStride(float *a, float *b, float *c, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;  // Total threads in grid
    
    // Each thread processes elements: idx, idx+stride, idx+2*stride, ...
    for (int i = idx; i < n; i += stride) {
        c[i] = a[i] + b[i];
    }
}

int main() {
    int n = 1000000;  // 1 million elements
    size_t size = n * sizeof(float);
    
    // Allocate and initialize
    float *h_a = (float*)malloc(size);
    float *h_b = (float*)malloc(size);
    float *h_c = (float*)malloc(size);
    
    for (int i = 0; i < n; i++) {
        h_a[i] = 1.0f;
        h_b[i] = 2.0f;
    }
    
    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);
    
    // Method 1: Naive (need many blocks)
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    
    printf("=== Array size: %d elements ===\n\n", n);
    printf("Method 1: Naive (1 thread = 1 element)\n");
    printf("  Blocks needed: %d\n", blocksPerGrid);
    printf("  Total threads: %d\n\n", blocksPerGrid * threadsPerBlock);
    
    vectorAddNaive<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);
    cudaDeviceSynchronize();
    
    // Method 2: Grid-stride (can use fewer blocks!)
    int limitedBlocks = 128;  // Use only 128 blocks
    printf("Method 2: Grid-Stride Loop\n");
    printf("  Blocks used: %d (much less!)\n", limitedBlocks);
    printf("  Total threads: %d\n", limitedBlocks * threadsPerBlock);
    printf("  Elements per thread: ~%d\n\n", n / (limitedBlocks * threadsPerBlock));
    
    vectorAddGridStride<<<limitedBlocks, threadsPerBlock>>>(d_a, d_b, d_c, n);
    cudaDeviceSynchronize();
    
    // Verify
    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);
    bool correct = true;
    for (int i = 0; i < n; i++) {
        if (h_c[i] != 3.0f) { correct = false; break; }
    }
    printf("Result: %s\n", correct ? "✅ CORRECT" : "❌ WRONG");
    
    // Cleanup
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
    free(h_a); free(h_b); free(h_c);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 grid_stride.cu -o grid_stride && ./grid_stride

### 2D Thread Indexing

For 2D data like images and matrices, we use 2D grids and blocks with `dim3`:

```cpp
dim3 threadsPerBlock(16, 16);  // 16×16 = 256 threads per block
dim3 numBlocks((width + 15) / 16, (height + 15) / 16);

myKernel<<<numBlocks, threadsPerBlock>>>(...);
```

Inside the kernel:
```cpp
int col = blockIdx.x * blockDim.x + threadIdx.x;  // X dimension
int row = blockIdx.y * blockDim.y + threadIdx.y;  // Y dimension

// 2D to 1D conversion for array access (row-major)
int idx = row * width + col;
```

In [ ]:
%%writefile indexing_2d.cu
/**
 * 2D Thread Indexing for Matrices
 * 
 * Demonstrates how to use dim3 for 2D grids and blocks.
 */

#include <stdio.h>
#include <cuda_runtime.h>

#define WIDTH 8
#define HEIGHT 6

// Print which thread handles which matrix element
__global__ void print2DIndices() {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (row < HEIGHT && col < WIDTH) {
        int linearIdx = row * WIDTH + col;
        printf("Thread (%d,%d) in Block (%d,%d) → Matrix[%d][%d] → Linear %d\n",
               threadIdx.x, threadIdx.y,
               blockIdx.x, blockIdx.y,
               row, col, linearIdx);
    }
}

// Matrix addition: C = A + B
__global__ void matrixAdd(float *A, float *B, float *C, int width, int height) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (row < height && col < width) {
        int idx = row * width + col;
        C[idx] = A[idx] + B[idx];
    }
}

int main() {
    printf("=== 2D Thread Indexing Demo ===\n");
    printf("Matrix size: %d x %d\n\n", WIDTH, HEIGHT);
    
    // Use 4x4 blocks
    dim3 threadsPerBlock(4, 4);
    dim3 numBlocks((WIDTH + 3) / 4, (HEIGHT + 3) / 4);
    
    printf("Grid: %d x %d blocks\n", numBlocks.x, numBlocks.y);
    printf("Block: %d x %d threads\n\n", threadsPerBlock.x, threadsPerBlock.y);
    
    printf("Thread mapping (first 10 only):\n");
    printf("----------------------------\n");
    print2DIndices<<<numBlocks, threadsPerBlock>>>();
    cudaDeviceSynchronize();
    
    // Matrix addition example
    printf("\n=== Matrix Addition ===\n");
    
    int n = WIDTH * HEIGHT;
    size_t size = n * sizeof(float);
    
    float *h_A = (float*)malloc(size);
    float *h_B = (float*)malloc(size);
    float *h_C = (float*)malloc(size);
    
    // Initialize: A[i][j] = i, B[i][j] = j
    for (int i = 0; i < HEIGHT; i++) {
        for (int j = 0; j < WIDTH; j++) {
            h_A[i * WIDTH + j] = (float)i;
            h_B[i * WIDTH + j] = (float)j;
        }
    }
    
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);
    
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);
    
    matrixAdd<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, WIDTH, HEIGHT);
    cudaDeviceSynchronize();
    
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);
    
    // Display result (C should be i + j at each position)
    printf("Result C = A + B:\n");
    for (int i = 0; i < HEIGHT; i++) {
        for (int j = 0; j < WIDTH; j++) {
            printf("%4.0f ", h_C[i * WIDTH + j]);
        }
        printf("\n");
    }
    
    // Cleanup
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    free(h_A); free(h_B); free(h_C);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 indexing_2d.cu -o indexing_2d && ./indexing_2d

### CUDA C++ Summary

**1D Indexing Formula:**
```cpp
int idx = blockIdx.x * blockDim.x + threadIdx.x;
```

**2D Indexing Formulas:**
```cpp
int col = blockIdx.x * blockDim.x + threadIdx.x;
int row = blockIdx.y * blockDim.y + threadIdx.y;
int linearIdx = row * width + col;  // Row-major layout
```

**Grid-Stride Loop Pattern:**
```cpp
int idx = blockIdx.x * blockDim.x + threadIdx.x;
int stride = blockDim.x * gridDim.x;
for (int i = idx; i < n; i += stride) {
    // Process element i
}
```

**Key Points:**
- Always check bounds: `if (idx < n)` or `if (row < height && col < width)`
- Use `dim3` for multi-dimensional configurations
- Grid-stride loops handle any array size with any grid size
- Thread indices are unique across the entire grid

---

## Part 2: Python/Numba (Optional Comparison)

The following sections demonstrate the same concepts using Python and Numba's CUDA JIT compiler. This is helpful for rapid prototyping but remember that production CUDA code is typically C++.

In [ ]:
# ⚙️ Colab Setup Cell - Run this first!
import subprocess, sys
try:
    import google.colab
    print("🔧 Running on Google Colab - Installing dependencies...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
    print("✅ Setup complete!")
except ImportError:
    print("💻 Running locally - make sure you have: pip install numba numpy")

# Day 2: Thread Indexing Mastery

Yesterday you launched your first CUDA kernel. Today we'll master the art of **thread indexing** - how each thread knows which data element to process.

## Learning Objectives
- Understand 1D, 2D, and 3D thread indexing
- Master the relationship between blocks, threads, and global indices
- Implement grid-stride loops for handling any array size
- Apply indexing to real 2D problems (matrices, images)

---

## 1. Thread Hierarchy Recap

```
                           GRID
                ┌─────────────────────────┐
                │ Block   Block   Block   │
                │ (0,0)   (1,0)   (2,0)   │
                │                         │
                │ Block   Block   Block   │
                │ (0,1)   (1,1)   (2,1)   │
                └─────────────────────────┘
                        
                      Each Block:
                ┌─────────────────────────┐
                │ Thread Thread Thread    │
                │ (0,0)  (1,0)  (2,0)     │
                │                         │
                │ Thread Thread Thread    │
                │ (0,1)  (1,1)  (2,1)     │
                └─────────────────────────┘
```

**Built-in Variables (in Numba):**
- `cuda.threadIdx.x/y/z` - Thread index within block
- `cuda.blockIdx.x/y/z` - Block index within grid
- `cuda.blockDim.x/y/z` - Threads per block
- `cuda.gridDim.x/y/z` - Blocks in grid

In [ ]:
# Setup
import numpy as np
from numba import cuda
import math

print("CUDA available:", cuda.is_available())
print("Current device:", cuda.get_current_device().name.decode())

## 2. 1D Indexing: The Foundation

For 1D arrays, each thread needs a unique **global index**:

```
Global Index = blockIdx.x * blockDim.x + threadIdx.x

Example: 3 blocks × 4 threads/block = 12 threads

Block 0:  Thread 0  Thread 1  Thread 2  Thread 3
          idx=0     idx=1     idx=2     idx=3
          
Block 1:  Thread 0  Thread 1  Thread 2  Thread 3  
          idx=4     idx=5     idx=6     idx=7
          
Block 2:  Thread 0  Thread 1  Thread 2  Thread 3
          idx=8     idx=9     idx=10    idx=11
```

Let's visualize this:

In [ ]:
@cuda.jit
def print_1d_indices(output):
    """Each thread writes its indices to the output array"""
    # Get indices manually
    block_id = cuda.blockIdx.x
    thread_id = cuda.threadIdx.x
    block_size = cuda.blockDim.x
    
    # Calculate global index
    global_idx = block_id * block_size + thread_id
    
    # Or use the convenient helper:
    # global_idx = cuda.grid(1)
    
    if global_idx < output.shape[0]:
        # Store: [global_idx, block_id, thread_id]
        output[global_idx, 0] = global_idx
        output[global_idx, 1] = block_id
        output[global_idx, 2] = thread_id

# Launch with 3 blocks × 4 threads
threads_per_block = 4
blocks = 3
total_threads = blocks * threads_per_block

output = np.zeros((total_threads, 3), dtype=np.int32)
output_d = cuda.to_device(output)

print_1d_indices[blocks, threads_per_block](output_d)
result = output_d.copy_to_host()

print("1D Thread Indexing Visualization")
print("=" * 50)
print(f"Configuration: {blocks} blocks × {threads_per_block} threads/block")
print("-" * 50)
print(f"{'Global Idx':^12} | {'Block ID':^10} | {'Thread ID':^10}")
print("-" * 50)
for row in result:
    print(f"{row[0]:^12} | {row[1]:^10} | {row[2]:^10}")

## 3. Boundary Checking: When Threads > Elements

What happens when array size isn't a perfect multiple of block size?

```
Array size: 10 elements
Block size: 4 threads
Blocks needed: ceil(10/4) = 3
Total threads: 3 × 4 = 12

Thread indices: 0  1  2  3  4  5  6  7  8  9  10  11
Array elements: ✓  ✓  ✓  ✓  ✓  ✓  ✓  ✓  ✓  ✓   ✗   ✗
                                               ↑    ↑
                                          Out of bounds!
```

**Always add boundary checks!**

In [ ]:
@cuda.jit
def safe_square(input_arr, output_arr, n):
    """Square each element with proper boundary check"""
    idx = cuda.grid(1)
    
    # CRITICAL: Boundary check
    if idx < n:
        output_arr[idx] = input_arr[idx] ** 2

# Array that's NOT a multiple of block size
N = 1000
a = np.arange(N, dtype=np.float32)
b = np.zeros(N, dtype=np.float32)

threads_per_block = 256
blocks = math.ceil(N / threads_per_block)

print(f"Array size: {N}")
print(f"Threads per block: {threads_per_block}")
print(f"Blocks needed: {blocks}")
print(f"Total threads: {blocks * threads_per_block}")
print(f"Extra threads (idle): {blocks * threads_per_block - N}")

a_d = cuda.to_device(a)
b_d = cuda.to_device(b)

safe_square[blocks, threads_per_block](a_d, b_d, N)
result = b_d.copy_to_host()

# Verify
expected = a ** 2
print(f"\n✅ Correct: {np.allclose(result, expected)}")

## 4. Grid-Stride Loops: The Professional Pattern

What if your array has **billions** of elements but you can only launch millions of threads?

**Grid-stride loop**: Each thread processes multiple elements, striding by the total grid size.

```
Array: [0][1][2][3][4][5][6][7][8][9][10][11]...

Grid size: 4 threads

Thread 0: processes indices 0, 4, 8, 12, ...
Thread 1: processes indices 1, 5, 9, 13, ...
Thread 2: processes indices 2, 6, 10, 14, ...
Thread 3: processes indices 3, 7, 11, 15, ...
```

This pattern is **essential** for production code!

In [ ]:
@cuda.jit
def grid_stride_square(input_arr, output_arr, n):
    """Process arbitrary-sized arrays with grid-stride loop"""
    # Starting index for this thread
    idx = cuda.grid(1)
    
    # Total number of threads in the grid
    stride = cuda.gridsize(1)  # = blockDim.x * gridDim.x
    
    # Grid-stride loop: each thread handles multiple elements
    while idx < n:
        output_arr[idx] = input_arr[idx] ** 2
        idx += stride  # Jump to next element this thread handles

# Huge array with limited grid
N = 100_000_000  # 100 million elements!
a = np.random.randn(N).astype(np.float32)
b = np.zeros(N, dtype=np.float32)

# Fixed, reasonable grid size
threads_per_block = 256
blocks = 256  # Only 256 blocks, but handles 100M elements!

print(f"Array size: {N:,}")
print(f"Grid size: {blocks} blocks × {threads_per_block} threads = {blocks * threads_per_block:,} threads")
print(f"Elements per thread (average): {N / (blocks * threads_per_block):.1f}")

a_d = cuda.to_device(a)
b_d = cuda.to_device(b)

import time
start = time.perf_counter()
grid_stride_square[blocks, threads_per_block](a_d, b_d, N)
cuda.synchronize()
elapsed = time.perf_counter() - start

result = b_d.copy_to_host()
print(f"\n⏱️  Time: {elapsed*1000:.2f} ms")
print(f"✅ Correct: {np.allclose(result, a**2)}")

## 5. 2D Indexing: Matrices and Images

For 2D data (matrices, images), we use 2D thread blocks and grids:

```
Image (Height × Width):
┌───┬───┬───┬───┬───┐
│0,0│0,1│0,2│0,3│0,4│  Row 0
├───┼───┼───┼───┼───┤
│1,0│1,1│1,2│1,3│1,4│  Row 1
├───┼───┼───┼───┼───┤
│2,0│2,1│2,2│2,3│2,4│  Row 2
└───┴───┴───┴───┴───┘

Thread coordinates:
  row = blockIdx.y * blockDim.y + threadIdx.y
  col = blockIdx.x * blockDim.x + threadIdx.x
```

In [ ]:
@cuda.jit
def matrix_double_2d(matrix, rows, cols):
    """Double each element using 2D indexing"""
    # Get 2D thread coordinates
    col, row = cuda.grid(2)  # Returns (x, y) = (col, row)
    
    # Boundary check for both dimensions
    if row < rows and col < cols:
        matrix[row, col] *= 2

# Create a small matrix to visualize
rows, cols = 6, 8
matrix = np.arange(rows * cols, dtype=np.float32).reshape(rows, cols)

print("Original matrix:")
print(matrix)
print()

# 2D block configuration
threads_per_block_2d = (4, 4)  # 4×4 = 16 threads per block
blocks_per_grid_x = math.ceil(cols / threads_per_block_2d[0])
blocks_per_grid_y = math.ceil(rows / threads_per_block_2d[1])
blocks_per_grid_2d = (blocks_per_grid_x, blocks_per_grid_y)

print(f"Block size: {threads_per_block_2d}")
print(f"Grid size: {blocks_per_grid_2d}")

matrix_d = cuda.to_device(matrix)
matrix_double_2d[blocks_per_grid_2d, threads_per_block_2d](matrix_d, rows, cols)
result = matrix_d.copy_to_host()

print("\nDoubled matrix:")
print(result)

## 6. Practical Example: Image Processing (Grayscale)

Let's apply 2D indexing to convert an RGB image to grayscale.

Grayscale formula: `Y = 0.299*R + 0.587*G + 0.114*B`

In [ ]:
@cuda.jit
def rgb_to_grayscale(rgb_image, gray_image, height, width):
    """Convert RGB image to grayscale using 2D indexing"""
    col, row = cuda.grid(2)
    
    if row < height and col < width:
        # RGB channels
        r = rgb_image[row, col, 0]
        g = rgb_image[row, col, 1]
        b = rgb_image[row, col, 2]
        
        # Weighted sum (human perception weighting)
        gray = 0.299 * r + 0.587 * g + 0.114 * b
        
        gray_image[row, col] = gray

# Create a synthetic RGB image (like a gradient)
height, width = 1080, 1920  # Full HD
rgb_image = np.random.randint(0, 256, (height, width, 3), dtype=np.uint8).astype(np.float32)

gray_image = np.zeros((height, width), dtype=np.float32)

# Configure 2D grid
threads_per_block = (16, 16)  # 256 threads per block
blocks_x = math.ceil(width / threads_per_block[0])
blocks_y = math.ceil(height / threads_per_block[1])
blocks_per_grid = (blocks_x, blocks_y)

print(f"Image size: {height} × {width} ({height * width:,} pixels)")
print(f"Thread block: {threads_per_block}")
print(f"Grid: {blocks_per_grid}")
print(f"Total threads: {blocks_x * threads_per_block[0]} × {blocks_y * threads_per_block[1]}")

# Run on GPU
rgb_d = cuda.to_device(rgb_image)
gray_d = cuda.to_device(gray_image)

import time
start = time.perf_counter()
rgb_to_grayscale[blocks_per_grid, threads_per_block](rgb_d, gray_d, height, width)
cuda.synchronize()
gpu_time = time.perf_counter() - start

gray_result = gray_d.copy_to_host()

# Compare with NumPy (CPU)
start = time.perf_counter()
gray_cpu = 0.299 * rgb_image[:,:,0] + 0.587 * rgb_image[:,:,1] + 0.114 * rgb_image[:,:,2]
cpu_time = time.perf_counter() - start

print(f"\n⏱️  GPU time: {gpu_time*1000:.3f} ms")
print(f"⏱️  CPU time: {cpu_time*1000:.3f} ms")
print(f"🚀 Speedup: {cpu_time/gpu_time:.2f}x")
print(f"✅ Correct: {np.allclose(gray_result, gray_cpu, atol=1e-5)}")

## 🎯 Exercises

### Exercise 1: Manual Index Calculation
Implement 1D indexing WITHOUT using `cuda.grid()` - calculate manually using `blockIdx`, `blockDim`, `threadIdx`.

In [ ]:
# TODO Exercise 1: Manual indexing
@cuda.jit
def manual_index_add(a, b, c):
    """Add arrays using MANUAL index calculation"""
    # TODO: Calculate global index manually
    # idx = blockIdx.x * blockDim.x + threadIdx.x
    idx = 0  # FIX THIS
    
    if idx < c.size:
        c[idx] = a[idx] + b[idx]

# Test your implementation
# N = 1000
# a = np.random.randn(N).astype(np.float32)
# b = np.random.randn(N).astype(np.float32)
# ...

### Exercise 2: 2D Grid-Stride Loop
Implement a 2D grid-stride loop for processing very large images.

In [ ]:
# TODO Exercise 2: 2D Grid-Stride Loop
@cuda.jit
def grid_stride_2d(matrix, height, width):
    """Process large matrix with 2D grid-stride loop"""
    # Starting position
    start_col, start_row = cuda.grid(2)
    
    # Stride (grid size in each dimension)
    stride_col, stride_row = cuda.gridsize(2)
    
    # TODO: Implement 2D grid-stride loop
    # Hint: Use nested while loops
    # row = start_row
    # while row < height:
    #     col = start_col
    #     while col < width:
    #         # process matrix[row, col]
    #         col += stride_col
    #     row += stride_row
    pass

# Test with a huge matrix
# ...

### Exercise 3: Image Negative
Create a kernel that inverts an image (negative): `output[i,j] = 255 - input[i,j]`

In [ ]:
# TODO Exercise 3: Image Negative
@cuda.jit
def image_negative(input_img, output_img, height, width):
    """Invert image: output = 255 - input"""
    # TODO: Implement using 2D indexing
    pass

# Test your implementation
# ...

## 📝 Key Takeaways

### Today You Learned:

1. **1D Indexing Formula**:
   ```
   global_idx = blockIdx.x * blockDim.x + threadIdx.x
   # Or simply: cuda.grid(1)
   ```

2. **2D Indexing Formula**:
   ```
   col, row = cuda.grid(2)
   # Manually: 
   # row = blockIdx.y * blockDim.y + threadIdx.y
   # col = blockIdx.x * blockDim.x + threadIdx.x
   ```

3. **Always check boundaries**: `if idx < n:`

4. **Grid-stride loops** handle any array size with fixed grid:
   ```python
   idx = cuda.grid(1)
   stride = cuda.gridsize(1)
   while idx < n:
       # process element
       idx += stride
   ```

5. **Common block sizes**:
   - 1D: 256 or 512 threads
   - 2D: (16, 16) or (32, 32) threads

---

### 📚 Next Up: Day 3 - Memory Fundamentals
- cudaMalloc vs cudaMallocManaged
- Pinned vs pageable memory
- Memory transfer optimization

---

### 🔗 Resources
- [Thread Hierarchy - Programming Guide](../../cuda-programming-guide/01-introduction/programming-model.md)
- [Quick Reference](../../notes/cuda-quick-reference.md)